**University of Edinburgh**\
**School of Mathematics**\
**Bayesian Data Analysis, 2020/2021, Semester 2**\
**Daniel Paulin & Nicolò Margaritella**

**Solutions for Workshop 5: Bayesian methods for Hierarchical Models (HM)**


**The purpose of this Practical is to give you experience using Bayesian methods to fit Hierarchical
Models. The two data sets needed, `dyestuff.csv`, and `wartpid.csv`, are available on Learn but they will be automatically uploaded by the code below.**

# 1.  **Modelling yields of a dye from different input batches.**

**In chemical reactions, the yield measures the amount of reactants produced in a reaction (as
usually not 100\% of the reactants are converted to products following the stoichiometry of
the reaction). This dataset, `dyestuff.csv`, has 30 records with two fields, `yield` and `batch`.
Yield, the outcome variable, is grams of a "dyestuff" called Naphthalene Black 12B. The data
are the result of a study to see how variation between batches of an intermediate product for
the synthesis of the dyestuff, called H-acid, contributed to variation in the yield. Six batches,
labeled A, B, C, D, E, and F were randomly sampled at the works manufacture. From each
batch five preparations of the dyestuff were made at the laboratory, and then the yield was
measured.**


In [ ]:
system("wget --no-check-certificate -r 'https://drive.google.com/uc?export=download&id=1ZK2Ovpk4hbWJzlSrlXuRlg9SnSpbINUJ' -O /kaggle/working/Dyestuff.csv")
system("wget --no-check-certificate -r 'https://drive.google.com/uc?export=download&id=15fbshTQT3xoo1pS_xGL02zcG98Y_iqiJ' -O /kaggle/working/wartpid.csv")
# You need to enable the Internet in Settings in Kaggle (right hand side menu) before running this
dye.data <- read.csv("/kaggle/working/Dyestuff.csv",header=TRUE)

**(i) EDA: Produce side-by-side boxplots of the yields for each of the 5 batches (e.g. by using the functions `boxplot` and `split`). What patterns do you observe? What does the variation within each batch look like?**

In [ ]:
# Boxplot
boxplot(split(dye.data$Yield,dye.data$Batch),main="Dyestuff Yields")

**(ii) Fit the following non-hierarchical (Independent) model using JAGS:**

$$\text{yield}_{ji}\sim N(\theta_j,\sigma^2)\quad j=A,\dots,F$$

**This is simply a regression analysis with 5 indicator variables representing 5 intercepts.
First, you should recode your index j to a numeric scale by using:**

In [ ]:
dye.data$Batch <- as.numeric(as.factor(dye.data$Batch))

**Use the following normal priors for the $\theta$'s and uniform prior for $\sigma$ [1]:**

$$\theta_j \overset{iid}{\sim}  N(\mu_\theta = 1500, \sigma^2_\theta = 1000^2) \quad j = A, B, \dots, F$$

$$\sigma \sim \text{Unif}(0,700) $$

**Run 3 chains with initial values chosen randomly from the prior distributions.**

[1]As recommended by A. Gelman, Prior distributions for variance parameters in hierarchical models, Bayesian
Analysis 1(3):515-533, 2006.

In [ ]:
#This code loads a compiled version of JAGS and rjags from a zip file on Onedrive, and loads rjags. It should only take a few seconds.
#IMPORTANT: Go to the Kaggle Settings (right hand side click on K icon) and enable the Internet option in Settings before running this.
system("wget --no-check-certificate -r 'https://uoe-my.sharepoint.com/:u:/g/personal/dpaulin_ed_ac_uk/EX_-yUc-bIZJhLXHcZxpOj8Ba6dwC15X_MjYoox-xM2KlQ?download=1' -O /kaggle/working/kaggle_JAGS.zip")
system("unzip /kaggle/working/kaggle_JAGS.zip")
system("rm /kaggle/working/kaggle_JAGS.zip")
system("cd /kaggle/working/JAGS-4.3.0")
system("make install")
library(rjags,lib.loc="/kaggle/working")
#If it ran correctly, you should see 
#Loading required package: coda
#Linked to JAGS 4.3.0
#Loaded modules: basemod,bugs

#In case you are still experiencing difficulties with this, please use the following code (this compiles and installs JAGS from the source, it takes 6-7 minutes):
#system("wget https://sourceforge.net/projects/mcmc-jags/files/JAGS/4.x/Source/JAGS-4.3.0.tar.gz -P /kaggle/working")
#system("tar xvfz /kaggle/working/JAGS-4.3.0.tar.gz")
#system("cd /kaggle/working/JAGS-4.3.0")
#system("/kaggle/working/JAGS-4.3.0/configure")
#system("make")
#system("make install")
#install.packages("rjags", lib="/kaggle/working")
#library(rjags,lib.loc="/kaggle/working")

In [ ]:
set.seed(11)
require(rjags)
mu.theta <- 1500
sigma.theta <- 1000
sigma.ub <- 700
dye.data$j.batch <- as.numeric(as.factor(dye.data$Batch))
# data list
dye.fixed.data <- list(Yield=dye.data$Yield, n=dim(dye.data)[1],
j.batch=dye.data$j.batch, J=max(dye.data$j.batch),
mu.theta=mu.theta,sigma.theta=sigma.theta, sigma.ub=sigma.ub)
# initial values
dye.fixed.inits <- function(){
list(theta=rnorm(max(dye.data$j.batch), mu.theta, sigma.theta),
sigma=runif(1, 0,sigma.ub))
}
# Model
dye.fixed.model <- "model {
#likelihood
for(i in 1:n) {
Yield[i] ~ dnorm(theta[j.batch[i]], tau)
}
# priors for independent means
for(j in 1:J){
theta[j] ~ dnorm(mu.theta,tau.theta)
}
tau.theta <- 1/pow(sigma.theta,2)
# prior for variance of the observations
tau <- 1/pow(sigma,2)
sigma ~ dunif(0,sigma.ub)
}"
#
dye.fixed.res.A <- jags.model(file=textConnection(dye.fixed.model),
data=dye.fixed.data, inits=dye.fixed.inits, n.chains=3,
quiet = TRUE)
#
update(dye.fixed.res.A, n.iter=100)
#
dye.fixed.res.B <- coda.samples(dye.fixed.res.A,
variable.names=c("theta","sigma"), n.iter=5000)
#
summary(dye.fixed.res.B)

**(iii) choose enough burn-in and run length so that you are resonably confident that the chains have converged
(you can plot the chains and check the Gelman-Rubin plot to check that) and you get at least 5000 effective simulations for every parameter(use the `effectiveSize` function).**

In [ ]:
effectiveSize(dye.fixed.res.B)
## sigma    theta[1]  theta[2]  theta[3]  theta[4]  theta[5] theta[6]
## 6266.012 14797.205 15402.081 15073.129 16269.901 15000.000 15740.752
plot(dye.fixed.res.B)
gelman.plot(dye.fixed.res.B)
#gelman.diag(dye.fixed.res.B)
#autocorr.plot(dye.fixed.res.B)

**(iv) Compare the posterior means (use the summary command) to the ordinary sample
mean which can be found by:**

In [ ]:
sapply(split(dye.data$Yield,dye.data$Batch),mean)

In [ ]:
dye.fixed.summary <- summary(dye.fixed.res.B)
dye.fixed.summary$statistics[-1,"Mean"]

**<u>Solution</u>: the posterior expected values for the fixed effects are almost equal to the sample means.**

**(v) Compute the probability for each batch of having an expected yield greater than
1500gr according to the fixed effects model. Which batches appear to be significantly
over- or under-producing Naphthalene Black 12B in average? (Assume that the
desired yield is 1500gr). First, join all chains in one dataframe:**

In [ ]:
# Join all chains in one data.frame
dye.fixed.output <- do.call(rbind.data.frame, dye.fixed.res.B)

In [ ]:

mean(dye.fixed.output$`theta[1]`>1500)
## [1] 0.583
mean(dye.fixed.output$`theta[2]`>1500)
## [1] 0.890 over 
mean(dye.fixed.output$`theta[3]`>1500)
## [1] 0.995 over
mean(dye.fixed.output$`theta[4]`>1500)
## [1] 0.465
mean(dye.fixed.output$`theta[5]`>1500)
## [1] 1 over
mean(dye.fixed.output$`theta[6]`>1500)
## [1] 0.096 under

**<u>Solution</u>: according to the independent model, batches B, C and E are over-
producing in mean with a probability of approx. 89%, 99.5% and 100% respectively. Batch
F is under-producing with a probability of 90.4%.**




**(b i) Fit the following hierarchical model**

$$\text{yield}_{ji}\sim N(\theta_j,\sigma^2)\quad j=A,\dots,F$$

$$\theta_j \sim N(\mu_\theta , \sigma^2_\theta) \quad j = A, \dots, F$$

**Use these prior distributions for the hyper-parameters.**

$$ \sigma \sim \text{Unif}(0,700) $$

$$ \mu_\theta \sim N(2000,1000^2)$$

$$ \sigma_\theta \sim \text{Unif}(0,300) $$

- Choose 3 sets of initial values by randomly sampling from the prior distributions.
  Use a burn-in of 1000 and an inference run of 10 000.

- Calculate the Intraclass Correlation Coefficient (ICC) $\sigma^{2}_\theta/(\sigma^{2}_\theta +\sigma^{2})$ either by adding a
  line in the model code block or by computing the new variable with the chains after
  the inference process.

- When you execute the `coda.samples` function, have the posteriors for the 3 hyperparameters, the $\theta$'s and the ICC (if you are computing it inside the model) returned.

In [ ]:
set.seed(11)
require(rjags)
mu.mu.theta <- 2000
sigma.mu.theta <- 1000
sigma.theta.ub <- 300
sigma.ub <- 700
dye.hier.data <- list(Yield=dye.data$Yield, n=dim(dye.data)[1],
j.batch=dye.data$j.batch, J=max(dye.data$j.batch),
mu.mu.theta=mu.mu.theta, sigma.mu.theta=sigma.mu.theta,
sigma.theta.ub=sigma.theta.ub, sigma.ub=sigma.ub)
dye.hier.inits <- function(){
list(theta=rnorm(max(dye.data$j.batch), mu.theta, sigma.theta),
sigma=runif(1, 0,sigma.ub))
}
# Model
dye.hier.model <- "model{
#likelihood
for(i in 1:n){
Yield[i] ~ dnorm(theta[j.batch[i]], tau)
 }
# prior for dependent means
for(j in 1:J){
theta[j] ~ dnorm(mu.theta,tau.theta)
 }
# hyperpriors for random effect mean and variance
tau.theta <- 1/pow(sigma.theta,2)
sigma.theta ~ dunif(0, sigma.theta.ub)
tau.mu.theta <- 1/pow(sigma.mu.theta,2)
mu.theta ~ dnorm(mu.mu.theta, tau.mu.theta)
# prior for variance of the observations
tau <- 1/pow(sigma,2)
sigma ~ dunif(0, sigma.ub)
# Computed quantities
ICC <- pow(sigma.theta,2)/(pow(sigma.theta,2) + pow(sigma,2))
}"
#
dye.hier.res.A <- jags.model(file=textConnection(dye.hier.model),
data=dye.hier.data, inits=dye.hier.inits, n.chains=3,
quiet = TRUE)
#
update(dye.hier.res.A, n.iter=1000)
#
dye.hier.res.B <- coda.samples(dye.hier.res.A,
variable.names=c("theta","sigma","mu.theta","sigma.theta","ICC"),
n.iter=20000)

**(b ii) Carry out the following convergence diagnostic procedures: (1) trace plots (2) BGR, (3) autocorrelation
plots (4) effective sample size, and (5) the time series corrected MC standard error
divided by the standard deviations from the posterior for each parameter. If you are
satisfied regarding convergence, report the posterior means and standard deviations
for $\mu_\theta$, $\sigma_\theta$ and $\sigma$, else increase the burn-in and chain lengths and re-run.**

In [ ]:
# CHECKING THE CHAINS
#
# plot(dye.hier.res.B)
# gelman.diag(dye.hier.res.B)
# gelman.plot(dye.hier.res.B)
# autocorr.plot(dye.hier.res.B)
# effectiveSize(dye.hier.res.B)
dye.hier.summary <- summary(dye.hier.res.B)
dye.hier.summary$statistics
dye.hier.summary$statistics[, "Time-series SE"]/
dye.hier.summary$statistics[, "SD"] # MCMC error over SD of the parameters

**(b iii) Compute the probability for each batch of having an expected yield greater than 1500gr according to the hierarchical model and compare the results with the ones
for the Independent model.**

In [ ]:
#Join all chains in one data.frame
dye.hier.output <- do.call(rbind.data.frame, dye.hier.res.B)
# Fixed effects model, Pr(theta_j > 1500)
## 1) 0.583
## 2) 0.890
## 3) 0.995
## 4) 0.465
## 5) 1
## 6) 0.096
#
# Hierarchical model, Pr(theta_j > 1500)
mean(dye.hier.output$`theta[1]`>1500)
## [1] 0.684
mean(dye.hier.output$`theta[2]`>1500)
## [1] 0.907
mean(dye.hier.output$`theta[3]`>1500)
## [1] 0.994
mean(dye.hier.output$`theta[4]`>1500)
## [1] 0.583
mean(dye.hier.output$`theta[5]`>1500)
## [1] 1
mean(dye.hier.output$`theta[6]`>1500)
## [1] 0.228

**<u>Solution</u>: The major change is that the hierarchical model does not offer so high
probability of under-production of the chemical for batch F.**

**(b iv) Compare the posterior means (use the summary command)
to the ordinary sample mean, which can be found by:**

In [ ]:
sapply(split(dye.data$Yield,dye.data$Batch),mean)

In [ ]:
#sapply(split(dye.data$Yield,dye.data$Batch),mean)
## A    B   C     D    E    F
## 1505 1528 1564 1498 1600 1470
#
# dye.hier.summary <- summary(dye.hier.res.B)
dye.hier.summary$statistics[-(1:4),"Mean"]
#
#
mean(dye.hier.summary$statistics[-(1:4),"Mean"])
sd(dye.hier.summary$statistics[-(1:4),"Mean"])
# vs
mean(sapply(split(dye.data$Yield,dye.data$Batch),mean))
sd(sapply(split(dye.data$Yield,dye.data$Batch),mean))
#

**<u>Solution</u>: the posterior expected values for the hierarchical model are more similar
to the overall mean than the raw sample means.**

**(b v) What does the posterior for the ICC look like? What does its value mean?**

In [ ]:
# Join all chains in one data.frame
dye.hier.output <- do.call(rbind.data.frame, dye.hier.res.B)
# If ICC was not computed in the model:
#dye.hier.output$ICC <- dye.hier.output$sigma.theta^2 /
#( dye.hier.output$sigma.theta^2 + dye.hier.output$sigma^2 )
plot(density(dye.hier.output$ICC),main="ICC posterior density")
mean(dye.hier.output$ICC)

**<u>Solution</u>: The expected value of the ICC is approx. 0.5, which indicates that the variance
of the data is both observational and caused by the batches. The weight of these
two components is quite uncertain, though.**

# 2.  **Modelling the probability of genital warts and Pelvic inflammatory disease (PID).**

**Genital warts and Pelvic inflammatory disease (PID) are conditions that commonly
occur among adult women. These conditions are typically diagnosed after referral to and
consultation with a sexual health physician. A question of relevance to health service providers
is the extent to which there is clinically relevant variation between physicians in the frequency
with which PID and genital warts are diagnosed. The data set `wartpid.csv` is a 23 by 4
matrix that consists of records for 23 physicians (`doctor`), identified by a number only, all working at
the same Sexual Health Centre, the number of patients they saw (`consults`), the number of
cases of PID diagnosed (`PID`), and the number of cases of genital warts (`warts`) diagnosed.
Load the data into R.**

In [ ]:
system("wget --no-check-certificate -r 'https://drive.google.com/uc?export=download&id=15fbshTQT3xoo1pS_xGL02zcG98Y_iqiJ' -O /kaggle/working/wartpid.csv")
# You need to enable the Internet in Settings in Kaggle (right hand side menu) before running this
wartpid.data <- read.csv("wartpid.csv",header=TRUE)
head(wartpid.data)

**(i) Exploratory Data Analysis: Calculate the fraction of wart and fraction of PID diagnoses per patient (consult) for each physician and add them as new variables of the dataframe.**

**Produce the following 4 plots and put them on a single page (use the `par(mfrow=c(2,2))` command:**
- Barplot of warts fraction by physician (use the `barplot` function with an appropriate value for the `names.arg` argument)
- Barplot of PID fraction by physician.
- Barplot of consultations by physician.
- Scatterplot with smooth fit (`scatter.smooth()`) of wart fraction (Y axis) against PID (X axis) fraction.

In [ ]:
wartpid.data$PID.frac <- wartpid.data$PID / wartpid.data$consults
wartpid.data$warts.frac <- wartpid.data$warts / wartpid.data$consults
#
par(mfrow=c(2,2),oma=c(0,0,3,0))
barplot(wartpid.data$PID.frac, names.arg=wartpid.data$doctor,
main="PID fraction")
barplot(wartpid.data$warts.frac, names.arg=wartpid.data$doctor,
main="Warts fraction")
scatter.smooth(wartpid.data$warts.frac~wartpid.data$PID.frac,
ylab="Warts",xlab="PID", main="Warts vs PID")
barplot(wartpid.data$consults, names.arg=wartpid.data$doctor,
main = "Consultations" )
par(mfrow=c(1,1))

**(b i) <u>Identical logistic model</u> Fit a simple Bayesian model for the number of warts diagnoses where the probability of diagnose is the same for all physicians. You could set
a Beta(0.5; 0.5) prior for the probability $p$, but, in this case we are going to take another
approach and set a $N(0; 20^2)$ prior for the $\text{logit}(p) = \beta_0$**

**Compute the predictive distribution for replicates of the observations. You can do
this by duplicating the line where the likelihood is defined with a different name for
the response variable (e.g., `warts.pred[i] ~ dbinom(p, consults[i])`).**

In [ ]:
require(rjags)
# Data block
warts.ident.data <- list(n=dim(wartpid.data)[1], warts=wartpid.data$warts,
consults=wartpid.data$consults)
# Initial values
warts.ident.inits <- function(){ 
list(beta0=rnorm(1,0,10))
 }
# Model
warts.ident.model <- "model{
#likelihood
for(i in 1:n){
warts[i] ~ dbinom(p, consults[i])
warts.pred[i] ~ dbinom(p, consults[i])
 }
# prior for the probability
logit(p) <- beta0
beta0 ~ dnorm(0, 0.0025)
 }"
# Inference
warts.ident.res.A <- jags.model(file=textConnection(warts.ident.model),
data=warts.ident.data, inits=warts.ident.inits, n.chains=3,
quiet = TRUE)
#
update(warts.ident.res.A, n.iter=2000)
#
warts.ident.res.B <- coda.samples(warts.ident.res.A,
variable.names=c("beta0", "warts.pred", "p"), n.iter=10000)
summary(warts.ident.res.B)

**(b ii) Join all the chains in one dataframe. Remember that you can use `do.call(rbind.data.frame, warts.ident.res.B)`.**

**Plot the posterior density (`plot(density(...), xlim=c(0.01, 0.08))`) of the estimation for $p$ and then add points (using the `points` function) or vertical lines for all the observed proportions for the physicians. What do you observe? Do you think all the physicians diagnose the same proportion of warts? Do you think the identical model is good for this data?**

In [ ]:
warts.ident.output <- do.call(rbind.data.frame, warts.ident.res.B)
plot(density(warts.ident.output$p), main="p|y", xlim=c(0.01, 0.08))
points(wartpid.data$warts.frac, rep(0,23), col="firebrick2")

**(b iii) Another way of looking at the same problem. Plot the predictive distributions for
replicates of the observations with a line indicating the observed value (see code
below). What do you observe? Do you think the identical model is good for this
data?**

In [ ]:
# Plot all the discrete predictive distributions and the observed values
par(mfrow=c(6,4),mai=c(0.3,0.3,0.3,0.3))
for(i in 1:23){
auxtable <- table(warts.ident.output[,paste0("warts.pred[",i,"]")])
xaux <- as.numeric(names(auxtable))
plot(NA, xlim=c(min(xaux), max(xaux)), ylim=c(0,max(auxtable)),
xlab="x", ylab="y")
segments(x0=xaux, y0 = 0, x1=xaux, y1=auxtable)
abline(v=wartpid.data$warts[i], col="firebrick2", lwd=2)
 }
par(mfrow=c(1,1))

**<u>Solution</u>: For several physicians (e.g, physicians 8, 15, 19 and 21) the observed number of diagnoses falls in the tails of the predictive distributions. This model lacks some flexibility for the effect of the physician.**

**(b iv) Compute the predictive probability for physician 1 of observing less or equal diagnoses in the same amount of consults (considering that the probability of diagnose
stays the same). This is what is called the 'Bayesian p-value'.**

In [ ]:
mean(warts.ident.output$`warts.pred[1]`<=wartpid.data$warts[1])

**(c i) <u>Hierarchical logistic model</u>. Let us improve the previous model by including a random
effect of the physician on the probability of diagnosing warts. Set a prior distribution
$N(0; 10^2)$ for the mean of $\beta_0$ and a $\text{Unif}(0; 20)$ for its standard deviation.**

**Once again, do prediction for the replicates of the data. We are going to do predictions considering the particular $p_i$ estimated for each physician (you only have to
add the indexing `p[i]` to the code line you introduced on the identical model).**

In [ ]:
require(rjags)
# Data block
warts.hier.data <- list(n=dim(wartpid.data)[1], warts=wartpid.data$warts,
consults=wartpid.data$consults)
# Initial values
warts.hier.inits <- function(){ 
list(mu.beta0=rnorm(1,0,10), sigma.beta0=runif(1,0,20))
}
# Model
warts.hier.model <- "model{
#likelihood
for(i in 1:n){ 
warts[i] ~ dbinom(p[i], consults[i])
warts.pred[i] ~ dbinom(p[i], consults[i])
logit(p[i]) <- beta0[i]
beta0[i] ~ dnorm(mu.beta0, tau.beta0)
 }
# hyperpriors for beta0
mu.beta0 ~ dnorm(0, 0.01)
tau.beta0 <- pow(sigma.beta0,-2)
sigma.beta0 ~ dunif(0, 20)
 }"
# Inference
warts.hier.res.A <- jags.model(file=textConnection(warts.hier.model),
data=warts.hier.data, inits=warts.hier.inits, n.chains=3,
quiet = TRUE)
#
update(warts.hier.res.A, n.iter=2000)
#
warts.hier.res.B <- coda.samples(warts.hier.res.A,
variable.names=c("mu.beta0", "sigma.beta0", "beta0", "warts.pred", "p"),
n.iter=10000)
#
summary(warts.hier.res.B)

**(c ii) Join all the chains in one dataframe and then plot the predictive distributions for replicates of the observations with a line indicating the observed value (see code below). What do you observe? Do you think
the hierarchical model provides a better fit for this data than the identical model?**

In [ ]:
# Join all the chains in one data.frame
warts.hier.output <- do.call(rbind.data.frame, warts.hier.res.B)
# Plot all the discrete predictive distributions and the observed values
par(mfrow=c(6,4),mai=c(0.3,0.3,0.3,0.3))
for(i in 1:23){ 
auxtable <- table(warts.hier.output[,paste0("warts.pred[",i,"]")])
xaux <- as.numeric(names(auxtable))
plot(NA, xlim=c(min(xaux), max(xaux)), ylim=c(0,max(auxtable)),
xlab="x", ylab="y")
segments(x0=xaux, y0 = 0, x1=xaux, y1=auxtable)
abline(v=wartpid.data$warts[i], col="firebrick2", lwd=2)
 }
par(mfrow=c(1,1))

**<u>Solution</u>: We observe that the prediction fits better the data using the hierarchical
model, as this model can count for the effect of the physician on the proportion of
diagnoses.**

**(c iii) Compute the predictive probability for physician 1 of observing less or equal diagnoses in the same amount of consults and compare it with the one of the Identical model. Can you explain the difference?**

In [ ]:
mean(warts.ident.output$`warts.pred[1]`<=wartpid.data$warts[1])
#0.09587
mean(warts.hier.output$`warts.pred[1]`<=wartpid.data$warts[1])

The probability of at least as many warts as observed by physician 1 is 0.094 according to the identical model, while 0.196 according to the hierarchical model. This seems to suggest that the observation is more reasonable according to the hierarchical model, so that model fits this data point better.

**(d) Implement the identical and hierarchical logistic models for this dataset in INLA. Compare the model fits using Bayesian model comparison criteria (log marginal likelihood, DIC and NLSCPO) and also by implementing the posterior predictive checks of (b iii) and (c iii) in INLA.**

In [ ]:
#This code unzips an installation of R-INLA from an online source, and loads INLA
#IMPORTANT: Go to the Kaggle Settings (right hand side click on K icon) and enable the Internet option in Settings before running this.
system("wget --no-check-certificate -r 'https://uoe-my.sharepoint.com/:u:/g/personal/dpaulin_ed_ac_uk/EUNBvDg_EJVFqSZJA3Xz7LsB5cVgqYk0HWWnOp74_Dr28A?download=1' -O /kaggle/working/kaggle_INLA.zip")
system("unzip /kaggle/working/kaggle_INLA.zip")
system("rm /kaggle/working/kaggle_INLA.zip")
library(INLA,lib.loc="/kaggle/working")
#If INLA has been successfully loaded, you should see the following:
#This is INLA_20.03.17 built 2021-01-02 20:27:47 UTC.
#See www.r-inla.org/contact-us for how to get help.
#To enable PARDISO sparse library; see inla.pardiso()

#The following code does the full installation. You can try it if the previous code fails, but this takes longer.
#install.packages("INLA",repos=c(getOption("repos"),INLA="https://inla.r-inla-download.org/R/stable"), dep=TRUE,lib="/kaggle/working")
#library(INLA,lib.loc="/kaggle/working")

First, we implement the identical model from part (b). 

In this case, no random effects are needed. The model corresponds to binomial likelihood in INLA, number of consultations per doctor is passed along to INLA using the `Ntrials` parameter (see the code below for more details).

In [ ]:
prior.beta <- list(mean.intercept = 0, prec.intercept = 1/(20^2))
m.warts.identical=inla(warts~1,data=wartpid.data,family="binomial",
                       control.family=list(link="logit"),
                       Ntrials=consults,
                       control.fixed=prior.beta,
                       control.compute=list(config=TRUE,cpo=TRUE,dic=TRUE))
#Including control.compute=list(config=TRUE,cpo=TRUE,dic=TRUE
#allows for posterior sampling, and computes CPO and DIC values

summary(m.warts.identical)

The summary statistics of the intercept $\beta_0$ are essentially identical to what we got from JAGS.

In [ ]:
prior.beta <- list(mean.intercept = 0, prec.intercept = 1/(10^2))

sigma.unif.prior.random.eff = "expression:
  b = 20;
  log_dens = (theta>=(-2*log(b)))*(-log(b)-theta/2-log(2)) + (theta<(-2*log(b)))*(-Inf);
  return(log_dens);"

b=20;
prec.prior.random.eff <- list(prec=list(prior = sigma.unif.prior.random.eff,
                                        initial = (-2*log(b)+1), fixed = FALSE))


m.warts.hierarchical=inla(warts~1+f(doctor,model="iid",hyper=prec.prior.random.eff),
                       data=wartpid.data,family="binomial",
                       control.family=list(link="logit"),
                       Ntrials=consults,
                       control.inla = list(strategy = "laplace", npoints = 40),
                       control.fixed=prior.beta,
                       control.compute=list(config=TRUE,cpo=TRUE,dic=TRUE))
#We have added control.inla = list(strategy = "laplace", npoints = 40)
#to increase the number of integration points,
#which helps with more accurate estimation of tail probabilities 
#that are used in CPO calculations
#Including control.compute=list(config=TRUE,cpo=TRUE,dic=TRUE
#allows for posterior sampling, and computes CPO and DIC values

summary(m.warts.hierarchical)

Now we compare the models using several Bayesian model comparison criteria.

In [ ]:
cat("Marginal log-likelihood of model 1:",m.warts.identical$mlik[1],"\n")
cat("Marginal log-likelihood of model 2:",m.warts.hierarchical$mlik[1],"\n")

cat("DIC of model 1:",m.warts.identical$dic$dic,"\n")
cat("DIC of model 2:",m.warts.hierarchical$dic$dic,"\n")

cat("NSLCPO of model 1:",-sum(log(m.warts.identical$cpo$cpo)),"\n")
cat("NSLCPO of model 2:",-sum(log(m.warts.hierarchical$cpo$cpo)),"\n")

As we can see, the hierarchical model has the best performance according to all 3 criteria.

Now we do the posterior predictive checks.
As the Predictor variables in the posterior samples of INLA contain the linear predictors ($\eta_i=\text{logit}(p_i)$), in order to get the posterior predictive samples, we need to apply the inverse logit function on these linear predictors, and then sample from the Binomial distribution taking into account the appropriate number of consultations. The logit link function is of the form
$$\text{logit}(p)=\log(p/(1-p))=\log(1-1/(1-p))$$,
and so the inverse logit function is
$$\text{inv.logit}(x)=1/(1+\exp(-x))$$.

In [ ]:
inv.logit <- function(x) {
  return(1/(1+exp(-x)))
}

nbsamp=10000
n=nrow(wartpid.data)
yrep1=matrix(0,nrow=n,ncol=nbsamp)
yrep2=matrix(0,nrow=n,ncol=nbsamp)

m.warts.identical.samples=inla.posterior.sample(n=nbsamp, result=m.warts.identical)
m.warts.hierarchical.samples=inla.posterior.sample(n=nbsamp, result=m.warts.hierarchical)
predictor.samples.identical=inla.posterior.sample.eval(function(...) {Predictor},
m.warts.identical.samples)
predictor.samples.hierarchical=inla.posterior.sample.eval(function(...) {Predictor},
m.warts.hierarchical.samples)

for (row.num in 1:n){   
yrep1[row.num,]<- rbinom(n=nbsamp, size=wartpid.data$consults[row.num],
                         prob=inv.logit(predictor.samples.identical[row.num,]))
yrep2[row.num,]<- rbinom(n=nbsamp, size=wartpid.data$consults[row.num],
                         prob=inv.logit(predictor.samples.hierarchical[row.num,]))    
}

The final step is to plot these replicated against the true values from the data. This is done in the same way as previously.

In [ ]:
par(mfrow=c(6,4),mai=c(0.3,0.3,0.3,0.3))
for(i in 1:23){ 
auxtable <- table(yrep1[i,])
xaux <- as.numeric(names(auxtable))
plot(NA, xlim=c(min(xaux), max(xaux)), ylim=c(0,max(auxtable)),
xlab="x", ylab="y")
segments(x0=xaux, y0 = 0, x1=xaux, y1=auxtable)
abline(v=wartpid.data$warts[i], col="firebrick2", lwd=2)
 }
par(mfrow=c(1,1))

In [ ]:
par(mfrow=c(6,4),mai=c(0.3,0.3,0.3,0.3))
for(i in 1:23){ 
auxtable <- table(yrep2[i,])
xaux <- as.numeric(names(auxtable))
plot(NA, xlim=c(min(xaux), max(xaux)), ylim=c(0,max(auxtable)),
xlab="x", ylab="y")
segments(x0=xaux, y0 = 0, x1=xaux, y1=auxtable)
abline(v=wartpid.data$warts[i], col="firebrick2", lwd=2)
 }
par(mfrow=c(1,1))

The plots for posterior predictive model checks are identical to what we got from JAGS. The hierarchical model has significantly better fit since all of the actual values from the data lie in the typical range of the replicates (while for the first model, there are a few of them that are very unlikely according to the replicates). The hierarchical model can account for the effect of the physician on the proportion of diagnoses, which explains the better fit.